# Part 1: Data Preparation
My goal in this stage is to load the data from each police force, clean each one so that they all have the same format, and merge all into one dataframe.

In [1]:
# import libraries
import pandas as pd
import numpy as np

I have previously retrieved the data from police websites and manually transfered it to csv files. Now I just need to load the data.

For those interested, the websites are:<br>
VPD: https://vancouver.ca/police/organization/planning-research-audit/neighbourhood-statistics.html <br>
TPS: https://data.torontopolice.on.ca/pages/asr-archives<br>
SPVM: https://spvm.qc.ca/en/Pages/Discover-SPVM/Organization/Annual-Reports

In [2]:
# read data into dataframes
vpd_df = pd.read_csv("./data/vpd_data.csv")
tps_df = pd.read_csv("./data/tps_data.csv")
spvm_df = pd.read_csv("./data/spvm_data.csv")

let's examine them.

In [3]:
vpd_df.head()

,Neighbourhood,Sex Offences,Assaults,Robbery,B&E,Theft of MV,Theft from Auto,Theft<>$5K,Arson,Mischief,Offensive Weapons
0,Arbutus Ridge,5,17,1,78,13,91,93,7,33,0
1,Central_Business_District,166,1819,258,798,185,6221,4757,62,1580,463
2,Dunbar-Southlands,6,16,2,85,13,167,97,2,64,2
3,Fairview,26,119,13,317,48,851,918,7,254,17
4,Grandview-Woodland,31,236,35,292,122,673,525,9,334,21


In [4]:
tps_df.head()

,Unnamed: 0,Homicide,Unnamed: 2,Unnamed: 3,Assault,Unnamed: 5,Unnamed: 6,Attempted Murder,Unnamed: 8,Unnamed: 9,...,Unnamed: 30,Offensive Weapons,Unnamed: 32,Unnamed: 33,Controlled Drugs and Substances Act,Unnamed: 35,Unnamed: 36,Other Federal Statute Violations,Unnamed: 38,Unnamed: 39
0,Division,Occurred,Cleared,% Cleared,Occurred,Cleared,% Cleared,Reported,Cleared,% Cleared,...,% Cleared,Reported,Cleared,% Cleared,Reported,Cleared,% Cleared,Reported,Cleared,% Cleared
1,D11,4,2,50.00%,701,480,68.50%,0,0,-,...,32.10%,85,75,88.20%,89,86,96.60%,9,7,77.80%
2,D12,9,6,66.70%,741,555,74.90%,23,15,65.20%,...,47.10%,84,77,91.70%,96,94,97.90%,20,19,95.00%
3,D13,4,1,25.00%,616,407,66.10%,9,3,33.30%,...,28.30%,51,43,84.30%,46,46,100.00%,7,7,100.00%
4,D14,4,4,100.00%,"1,423",888,62.40%,4,4,100.00%,...,18.10%,91,81,89.00%,149,137,91.90%,10,10,100.00%


^This one doesn't look too good.

In [5]:
spvm_df.head()

,Neighbourhood station,Assault,Sexual assaults,Robberies,Breaking and entering,Auto theft,Petty larceny,Firearms offences,Controlled Drugs and Substances Act
0,1.0,81,22.0,17.0,69.0,66.0,202,3.0,16.0
1,3.0,394,50.0,40.0,175.0,86.0,286,6.0,33.0
2,4.0,167,25.0,31.0,74.0,60.0,190,2.0,14.0
3,5.0,284,44.0,50.0,188.0,175.0,603,7.0,23.0
4,7.0,440,67.0,73.0,271.0,377.0,756,5.0,34.0


🤠 __Clean up the tps_df.__<br>
Only include the 'cleared' column from each crime type

In [6]:
# want to replace the 'cleared' column with the name of the crime

# first create list crime_type
crime_type = tps_df.columns.tolist()
# remove the elements that are not a crime type 
for i in crime_type:
    if 'Unnamed' in i:
        crime_type.remove(i)
for i in crime_type:
    if 'Unnamed' in i:
        crime_type.remove(i)
crime_type

['Homicide',
 'Assault',
 'Attempted Murder',
 'Robbery',
 'Sexual Violation',
 'Auto Theft',
 'Break & Enter',
 'Fraud',
 'Theft Over $5000',
 'Theft Under $5000',
 'Offensive Weapons',
 'Controlled Drugs and Substances Act',
 'Other Federal Statute Violations']

In [7]:
# create an index list (ind) to save the positions where we want to insert the crime type elements
header = tps_df.iloc[0,:].tolist()
ind=[]
for index, element in enumerate(header):
    if element == 'Cleared':
        ind.append(index)
print(ind)
print(header)

[2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]
['Division', 'Occurred', 'Cleared', '% Cleared', 'Occurred', 'Cleared', '% Cleared', 'Reported', 'Cleared', '% Cleared', 'Reported', 'Cleared', '% Cleared', 'Reported', 'Cleared', '% Cleared', 'Reported', 'Cleared', '% Cleared', 'Reported', 'Cleared', '% Cleared', 'Reported', 'Cleared', '% Cleared', 'Reported', 'Cleared', '% Cleared', 'Reported', 'Cleared', '% Cleared', 'Reported', 'Cleared', '% Cleared', 'Reported', 'Cleared', '% Cleared', 'Reported', 'Cleared', '% Cleared']


In [8]:
# create header list with "cleared" replaced by the appropriate crime type
n=0
for i in ind:
    header[i] = crime_type[n]
    n+=1
header

['Division',
 'Occurred',
 'Homicide',
 '% Cleared',
 'Occurred',
 'Assault',
 '% Cleared',
 'Reported',
 'Attempted Murder',
 '% Cleared',
 'Reported',
 'Robbery',
 '% Cleared',
 'Reported',
 'Sexual Violation',
 '% Cleared',
 'Reported',
 'Auto Theft',
 '% Cleared',
 'Reported',
 'Break & Enter',
 '% Cleared',
 'Reported',
 'Fraud',
 '% Cleared',
 'Reported',
 'Theft Over $5000',
 '% Cleared',
 'Reported',
 'Theft Under $5000',
 '% Cleared',
 'Reported',
 'Offensive Weapons',
 '% Cleared',
 'Reported',
 'Controlled Drugs and Substances Act',
 '% Cleared',
 'Reported',
 'Other Federal Statute Violations',
 '% Cleared']

In [9]:
# replace the header in the dataframe
tps_df.columns=header
tps_df

,Division,Occurred,Homicide,% Cleared,Occurred,Assault,% Cleared,Reported,Attempted Murder,% Cleared,...,% Cleared,Reported,Offensive Weapons,% Cleared,Reported,Controlled Drugs and Substances Act,% Cleared,Reported,Other Federal Statute Violations,% Cleared
0,Division,Occurred,Cleared,% Cleared,Occurred,Cleared,% Cleared,Reported,Cleared,% Cleared,...,% Cleared,Reported,Cleared,% Cleared,Reported,Cleared,% Cleared,Reported,Cleared,% Cleared
1,D11,4,2,50.00%,701,480,68.50%,0,0,-,...,32.10%,85,75,88.20%,89,86,96.60%,9,7,77.80%
2,D12,9,6,66.70%,741,555,74.90%,23,15,65.20%,...,47.10%,84,77,91.70%,96,94,97.90%,20,19,95.00%
3,D13,4,1,25.00%,616,407,66.10%,9,3,33.30%,...,28.30%,51,43,84.30%,46,46,100.00%,7,7,100.00%
4,D14,4,4,100.00%,"1,423",888,62.40%,4,4,100.00%,...,18.10%,91,81,89.00%,149,137,91.90%,10,10,100.00%
5,D22,5,1,20.00%,916,601,65.60%,9,7,77.80%,...,26.10%,96,92,95.80%,108,98,90.70%,12,8,66.70%
6,D23,8,4,50.00%,872,556,63.80%,6,4,66.70%,...,20.40%,60,53,88.30%,80,68,85.00%,8,6,75.00%
7,D31,8,6,75.00%,"1,396",976,69.90%,17,10,58.80%,...,33.70%,166,140,84.30%,78,72,92.30%,26,22,84.60%
8,D32,0,0,-,"1,204",771,64.00%,10,7,70.00%,...,24.80%,119,101,84.90%,53,48,90.60%,11,9,81.80%
9,D33,3,1,33.30%,720,479,66.50%,9,4,44.40%,...,23.20%,51,42,82.40%,40,34,85.00%,2,2,100.00%


In [10]:
# Remove unnecessary columns and rows

# only keep the columns with crime types and the division column
ind= [0] + ind
tps_df = tps_df.iloc[:,ind]

# only keep the rows with data and reset index afterwards
tps_df = tps_df.iloc[1:17,:]
tps_df.reset_index(drop=True, inplace=True)
tps_df

,Division,Homicide,Assault,Attempted Murder,Robbery,Sexual Violation,Auto Theft,Break & Enter,Fraud,Theft Over $5000,Theft Under $5000,Offensive Weapons,Controlled Drugs and Substances Act,Other Federal Statute Violations
0,D11,2,480,0,71,67,41,88,76,11,852,75,86,7
1,D12,6,555,15,43,79,93,65,125,16,707,77,94,19
2,D13,1,407,3,65,64,53,124,70,5,458,43,46,7
3,D14,4,888,4,73,115,43,199,240,21,777,81,137,10
4,D22,1,601,7,81,81,32,115,138,15,837,92,98,8
5,D23,4,556,4,89,96,52,44,111,17,439,53,68,6
6,D31,6,976,10,106,135,60,113,79,21,795,140,72,22
7,D32,0,771,7,63,96,68,71,150,25,"1,062",101,48,9
8,D33,1,479,4,40,59,42,25,64,12,504,42,34,2
9,D41,3,896,2,90,120,34,136,125,8,"1,301",119,88,10


Much better :)

🤠 __Make columns uniform across all three dataframes.__ <br>
I will only include columns that are common to all three police datasets. Upon inspection, these are:

Neighborhood<br>
Sex Offences<br>
Assaults<br>
Robbery<br>
Break & enter<br>
Auto Theft<br>
Other Theft<br>
Offensive Weapons<br><br>

Some columns will have to be combined, some renamed, and others dropped. 

In [11]:
# start with VPD
vpd_df

,Neighbourhood,Sex Offences,Assaults,Robbery,B&E,Theft of MV,Theft from Auto,Theft<>$5K,Arson,Mischief,Offensive Weapons
0,Arbutus Ridge,5,17,1,78,13,91,93,7,33,0
1,Central_Business_District,166,1819,258,798,185,6221,4757,62,1580,463
2,Dunbar-Southlands,6,16,2,85,13,167,97,2,64,2
3,Fairview,26,119,13,317,48,851,918,7,254,17
4,Grandview-Woodland,31,236,35,292,122,673,525,9,334,21
5,Hastings-Sunrise,27,118,12,197,85,656,260,2,207,7
6,Kensington-Cedar_Cottage,27,155,24,206,86,652,436,7,263,22
7,Kerrisdale,3,18,3,138,23,150,101,4,46,3
8,Killarney,11,50,4,77,38,248,165,2,80,7
9,Kitsilano,20,92,10,254,61,612,546,9,212,9


In [12]:
# view data types in each column
vpd_df.dtypes

Neighbourhood        object
Sex Offences          int64
Assaults              int64
Robbery               int64
B&E                   int64
Theft of MV           int64
Theft from Auto       int64
Theft<>$5K            int64
Arson                 int64
Mischief              int64
Offensive Weapons     int64
dtype: object

looks good. We want number formats so that we can perform mathematical operations.

In [13]:
# combine columns for "Other Theft" category
col = vpd_df['Theft from Auto'] + vpd_df['Theft<>$5K']
vpd_df['Other Theft'] = col
vpd_df.head()

,Neighbourhood,Sex Offences,Assaults,Robbery,B&E,Theft of MV,Theft from Auto,Theft<>$5K,Arson,Mischief,Offensive Weapons,Other Theft
0,Arbutus Ridge,5,17,1,78,13,91,93,7,33,0,184
1,Central_Business_District,166,1819,258,798,185,6221,4757,62,1580,463,10978
2,Dunbar-Southlands,6,16,2,85,13,167,97,2,64,2,264
3,Fairview,26,119,13,317,48,851,918,7,254,17,1769
4,Grandview-Woodland,31,236,35,292,122,673,525,9,334,21,1198


In [14]:
# drop columns that we do not need anymore
vpd_df.drop(['Theft from Auto','Theft<>$5K','Arson','Mischief'], axis=1, inplace=True)
vpd_df.head()

,Neighbourhood,Sex Offences,Assaults,Robbery,B&E,Theft of MV,Offensive Weapons,Other Theft
0,Arbutus Ridge,5,17,1,78,13,0,184
1,Central_Business_District,166,1819,258,798,185,463,10978
2,Dunbar-Southlands,6,16,2,85,13,2,264
3,Fairview,26,119,13,317,48,17,1769
4,Grandview-Woodland,31,236,35,292,122,21,1198


In [15]:
# change the order of the columns and rename them
vpd_df = vpd_df[['Neighbourhood','Sex Offences','Assaults','Robbery','B&E','Theft of MV','Other Theft' ,'Offensive Weapons']]
col_names = ['Neighborhood','Sex Offences','Assaults','Robbery','Break & Enter','Auto Theft','Other Theft', 'Offensive Weapons']
vpd_df.columns = col_names
vpd_df.head()

,Neighborhood,Sex Offences,Assaults,Robbery,Break & Enter,Auto Theft,Other Theft,Offensive Weapons
0,Arbutus Ridge,5,17,1,78,13,184,0
1,Central_Business_District,166,1819,258,798,185,10978,463
2,Dunbar-Southlands,6,16,2,85,13,264,2
3,Fairview,26,119,13,317,48,1769,17
4,Grandview-Woodland,31,236,35,292,122,1198,21


VPD done

In [16]:
# TPS next
tps_df

,Division,Homicide,Assault,Attempted Murder,Robbery,Sexual Violation,Auto Theft,Break & Enter,Fraud,Theft Over $5000,Theft Under $5000,Offensive Weapons,Controlled Drugs and Substances Act,Other Federal Statute Violations
0,D11,2,480,0,71,67,41,88,76,11,852,75,86,7
1,D12,6,555,15,43,79,93,65,125,16,707,77,94,19
2,D13,1,407,3,65,64,53,124,70,5,458,43,46,7
3,D14,4,888,4,73,115,43,199,240,21,777,81,137,10
4,D22,1,601,7,81,81,32,115,138,15,837,92,98,8
5,D23,4,556,4,89,96,52,44,111,17,439,53,68,6
6,D31,6,976,10,106,135,60,113,79,21,795,140,72,22
7,D32,0,771,7,63,96,68,71,150,25,"1,062",101,48,9
8,D33,1,479,4,40,59,42,25,64,12,504,42,34,2
9,D41,3,896,2,90,120,34,136,125,8,"1,301",119,88,10


In [17]:
# view data types
tps_df.dtypes

Division                               object
Homicide                               object
Assault                                object
Attempted Murder                       object
Robbery                                object
Sexual Violation                       object
Auto Theft                             object
Break & Enter                          object
Fraud                                  object
Theft Over $5000                       object
Theft Under $5000                      object
Offensive Weapons                      object
Controlled Drugs and Substances Act    object
Other Federal Statute Violations       object
dtype: object

Need to convert object types to integer types

In [18]:
# For values in the thousands, there is a comma, which is annoying

# get rid of commas in the data
def no_commas(list):
    '''
    takes a list of strings consisting of numbers, get rid of commas in every element of list and returns the list
    '''
    n=0 # keep track of index in list
    for element in list:
        if ',' in element:
            list[n] = element.replace(',','')
        n+=1
    return list

In [19]:
# Input columns of data with commas into no_commas() function
assault = no_commas(tps_df['Assault'].tolist())
theftunder5k = no_commas(tps_df['Theft Under $5000'].tolist())
theftunder5k

['852',
 '707',
 '458',
 '777',
 '837',
 '439',
 '795',
 '1062',
 '504',
 '1301',
 '626',
 '1082',
 '1072',
 '1729',
 '803',
 '1039']

looks good

In [20]:
# Replace those columns in the dataframe 
tps_df['Assault'] = pd.Series(assault)
tps_df['Theft Under $5000'] = pd.Series(theftunder5k)
tps_df

,Division,Homicide,Assault,Attempted Murder,Robbery,Sexual Violation,Auto Theft,Break & Enter,Fraud,Theft Over $5000,Theft Under $5000,Offensive Weapons,Controlled Drugs and Substances Act,Other Federal Statute Violations
0,D11,2,480,0,71,67,41,88,76,11,852,75,86,7
1,D12,6,555,15,43,79,93,65,125,16,707,77,94,19
2,D13,1,407,3,65,64,53,124,70,5,458,43,46,7
3,D14,4,888,4,73,115,43,199,240,21,777,81,137,10
4,D22,1,601,7,81,81,32,115,138,15,837,92,98,8
5,D23,4,556,4,89,96,52,44,111,17,439,53,68,6
6,D31,6,976,10,106,135,60,113,79,21,795,140,72,22
7,D32,0,771,7,63,96,68,71,150,25,1062,101,48,9
8,D33,1,479,4,40,59,42,25,64,12,504,42,34,2
9,D41,3,896,2,90,120,34,136,125,8,1301,119,88,10


In [21]:
# now that there are no commas, we can convert the data columns to integer type
A=tps_df[['Division']] # we want to keep Division as object type
B=tps_df.iloc[:,1:].astype('int64') # change the rest to integer type

# check results
print(A.dtypes)
print(B.dtypes)

Division    object
dtype: object
Homicide                               int64
Assault                                int64
Attempted Murder                       int64
Robbery                                int64
Sexual Violation                       int64
Auto Theft                             int64
Break & Enter                          int64
Fraud                                  int64
Theft Over $5000                       int64
Theft Under $5000                      int64
Offensive Weapons                      int64
Controlled Drugs and Substances Act    int64
Other Federal Statute Violations       int64
dtype: object


In [22]:
# merge columns back into one dataframe
tps_df = pd.merge(A, B, left_index=True, right_index=True)
tps_df.dtypes

Division                               object
Homicide                                int64
Assault                                 int64
Attempted Murder                        int64
Robbery                                 int64
Sexual Violation                        int64
Auto Theft                              int64
Break & Enter                           int64
Fraud                                   int64
Theft Over $5000                        int64
Theft Under $5000                       int64
Offensive Weapons                       int64
Controlled Drugs and Substances Act     int64
Other Federal Statute Violations        int64
dtype: object

success!

In [23]:
# now we can go ahead with combining columns
col = tps_df['Theft Over $5000'] + tps_df['Theft Under $5000']
tps_df['Other Theft'] = col
tps_df.head()

,Division,Homicide,Assault,Attempted Murder,Robbery,Sexual Violation,Auto Theft,Break & Enter,Fraud,Theft Over $5000,Theft Under $5000,Offensive Weapons,Controlled Drugs and Substances Act,Other Federal Statute Violations,Other Theft
0,D11,2,480,0,71,67,41,88,76,11,852,75,86,7,863
1,D12,6,555,15,43,79,93,65,125,16,707,77,94,19,723
2,D13,1,407,3,65,64,53,124,70,5,458,43,46,7,463
3,D14,4,888,4,73,115,43,199,240,21,777,81,137,10,798
4,D22,1,601,7,81,81,32,115,138,15,837,92,98,8,852


In [24]:
# drop columns
tps_df.drop(['Homicide','Attempted Murder',
             'Fraud','Theft Over $5000',
             'Theft Under $5000', 
             'Controlled Drugs and Substances Act',
             'Other Federal Statute Violations'],
           axis=1, inplace=True)
tps_df.head()

,Division,Assault,Robbery,Sexual Violation,Auto Theft,Break & Enter,Offensive Weapons,Other Theft
0,D11,480,71,67,41,88,75,863
1,D12,555,43,79,93,65,77,723
2,D13,407,65,64,53,124,43,463
3,D14,888,73,115,43,199,81,798
4,D22,601,81,81,32,115,92,852


In [25]:
# reorganize and rename columns
tps_df = tps_df[['Division',
                 'Sexual Violation',
                 'Assault', 'Robbery',
                 'Break & Enter',
                 'Auto Theft',
                 'Other Theft',
                 'Offensive Weapons']]
tps_df.columns = col_names
tps_df

,Neighborhood,Sex Offences,Assaults,Robbery,Break & Enter,Auto Theft,Other Theft,Offensive Weapons
0,D11,67,480,71,88,41,863,75
1,D12,79,555,43,65,93,723,77
2,D13,64,407,65,124,53,463,43
3,D14,115,888,73,199,43,798,81
4,D22,81,601,81,115,32,852,92
5,D23,96,556,89,44,52,456,53
6,D31,135,976,106,113,60,816,140
7,D32,96,771,63,71,68,1087,101
8,D33,59,479,40,25,42,516,42
9,D41,120,896,90,136,34,1309,119


TPS done

In [26]:
# SPVM
spvm_df

,Neighbourhood station,Assault,Sexual assaults,Robberies,Breaking and entering,Auto theft,Petty larceny,Firearms offences,Controlled Drugs and Substances Act
0,1.0,81,22.0,17.0,69.0,66.0,202,3.0,16.0
1,3.0,394,50.0,40.0,175.0,86.0,286,6.0,33.0
2,4.0,167,25.0,31.0,74.0,60.0,190,2.0,14.0
3,5.0,284,44.0,50.0,188.0,175.0,603,7.0,23.0
4,7.0,440,67.0,73.0,271.0,377.0,756,5.0,34.0
...,...,...,...,...,...,...,...,...,...
114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# data types
spvm_df.dtypes

Neighbourhood station                  float64
Assault                                 object
Sexual assaults                        float64
Robberies                              float64
Breaking and entering                  float64
Auto theft                             float64
Petty larceny                           object
Firearms offences                      float64
Controlled Drugs and Substances Act    float64
dtype: object

In [28]:
# change the two object type columns into float64 like the rest
# first need to get rid of commas
# convert to strings in order to use no_commas function
spvm_df['Assault'] = spvm_df['Assault'].astype('str')
spvm_df['Petty larceny'] = spvm_df['Petty larceny'].astype('str')

In [29]:
# remove commas
assault = no_commas(spvm_df['Assault'].tolist())
pettylarceny = no_commas(spvm_df['Petty larceny'].tolist())
pettylarceny

['202',
 '286',
 '190',
 '603',
 '756',
 '411',
 '179',
 '494',
 '724',
 '1389',
 '150',
 '675',
 '798',
 '500',
 '2687',
 '2343',
 '814',
 '173',
 '452',
 '686',
 '581',
 '465',
 '213',
 '699',
 '2025',
 '870',
 '667',
 '747',
 '787',
 '315',
 '590',
 '934',
 '474',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan']

In [30]:
# add back into dataframe whiile changing data type to float64
spvm_df['Assault'] = pd.Series(assault).astype('float64')
spvm_df['Petty larceny'] = pd.Series(pettylarceny).astype('float64')
spvm_df

,Neighbourhood station,Assault,Sexual assaults,Robberies,Breaking and entering,Auto theft,Petty larceny,Firearms offences,Controlled Drugs and Substances Act
0,1.0,81.0,22.0,17.0,69.0,66.0,202.0,3.0,16.0
1,3.0,394.0,50.0,40.0,175.0,86.0,286.0,6.0,33.0
2,4.0,167.0,25.0,31.0,74.0,60.0,190.0,2.0,14.0
3,5.0,284.0,44.0,50.0,188.0,175.0,603.0,7.0,23.0
4,7.0,440.0,67.0,73.0,271.0,377.0,756.0,5.0,34.0
...,...,...,...,...,...,...,...,...,...
114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# check types again
spvm_df.dtypes

Neighbourhood station                  float64
Assault                                float64
Sexual assaults                        float64
Robberies                              float64
Breaking and entering                  float64
Auto theft                             float64
Petty larceny                          float64
Firearms offences                      float64
Controlled Drugs and Substances Act    float64
dtype: object

Nice

In [32]:
# get rid of NaN rows
# first identify where they are
spvm_df.loc[np.isnan(spvm_df['Assault'])].index

Int64Index([ 33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
             46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
             59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
             72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
             85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
             98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
            111, 112, 113, 114, 115, 116, 117, 118],
           dtype='int64')

They start from the 33rd row and continue to the end

In [33]:
# only keep rows with data
spvm_df = spvm_df.iloc[:33,:]
spvm_df

,Neighbourhood station,Assault,Sexual assaults,Robberies,Breaking and entering,Auto theft,Petty larceny,Firearms offences,Controlled Drugs and Substances Act
0,1.0,81.0,22.0,17.0,69.0,66.0,202.0,3.0,16.0
1,3.0,394.0,50.0,40.0,175.0,86.0,286.0,6.0,33.0
2,4.0,167.0,25.0,31.0,74.0,60.0,190.0,2.0,14.0
3,5.0,284.0,44.0,50.0,188.0,175.0,603.0,7.0,23.0
4,7.0,440.0,67.0,73.0,271.0,377.0,756.0,5.0,34.0
5,8.0,522.0,58.0,52.0,148.0,69.0,411.0,10.0,36.0
6,9.0,89.0,11.0,15.0,68.0,45.0,179.0,2.0,8.0
7,11.0,447.0,44.0,83.0,174.0,88.0,494.0,10.0,48.0
8,13.0,527.0,62.0,77.0,211.0,99.0,724.0,4.0,32.0
9,26.0,660.0,82.0,144.0,363.0,258.0,1389.0,13.0,86.0


In [34]:
# drop columns
spvm_df.drop(['Controlled Drugs and Substances Act'], axis=1, inplace=True)
spvm_df.head()

C:\Users\jasmi\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Neighbourhood station,Assault,Sexual assaults,Robberies,Breaking and entering,Auto theft,Petty larceny,Firearms offences
0,1.0,81.0,22.0,17.0,69.0,66.0,202.0,3.0
1,3.0,394.0,50.0,40.0,175.0,86.0,286.0,6.0
2,4.0,167.0,25.0,31.0,74.0,60.0,190.0,2.0
3,5.0,284.0,44.0,50.0,188.0,175.0,603.0,7.0
4,7.0,440.0,67.0,73.0,271.0,377.0,756.0,5.0


In [35]:
# reorganize and rename columns
spvm_df = spvm_df[['Neighbourhood station',
                  'Sexual assaults',
                  'Assault',
                  'Robberies',
                  'Breaking and entering',
                  'Auto theft',
                  'Petty larceny',
                  'Firearms offences']]
spvm_df.columns = col_names
spvm_df

,Neighborhood,Sex Offences,Assaults,Robbery,Break & Enter,Auto Theft,Other Theft,Offensive Weapons
0,1.0,22.0,81.0,17.0,69.0,66.0,202.0,3.0
1,3.0,50.0,394.0,40.0,175.0,86.0,286.0,6.0
2,4.0,25.0,167.0,31.0,74.0,60.0,190.0,2.0
3,5.0,44.0,284.0,50.0,188.0,175.0,603.0,7.0
4,7.0,67.0,440.0,73.0,271.0,377.0,756.0,5.0
5,8.0,58.0,522.0,52.0,148.0,69.0,411.0,10.0
6,9.0,11.0,89.0,15.0,68.0,45.0,179.0,2.0
7,11.0,44.0,447.0,83.0,174.0,88.0,494.0,10.0
8,13.0,62.0,527.0,77.0,211.0,99.0,724.0,4.0
9,26.0,82.0,660.0,144.0,363.0,258.0,1389.0,13.0


Great, now all three dataframes have the same columns in the same order

🤠 __Calculate crime rate.__ <br>
I want to normalize the crime data in each city for fair comparison. I will use crime rate rather than the crime occurrence number.<br><br>
As mentioned in the report, The rate for a specific crime type in a specific neighborhood will be calculated as follows:<br><br>
$$
\frac{\text{number of occurrences of that crime type in that neighborhood}}{\text{all occurrences of that crime across all neighborhoods in that municipality}}
$$

In [36]:
# start with VPD
vpd_df

,Neighborhood,Sex Offences,Assaults,Robbery,Break & Enter,Auto Theft,Other Theft,Offensive Weapons
0,Arbutus Ridge,5,17,1,78,13,184,0
1,Central_Business_District,166,1819,258,798,185,10978,463
2,Dunbar-Southlands,6,16,2,85,13,264,2
3,Fairview,26,119,13,317,48,1769,17
4,Grandview-Woodland,31,236,35,292,122,1198,21
5,Hastings-Sunrise,27,118,12,197,85,916,7
6,Kensington-Cedar_Cottage,27,155,24,206,86,1088,22
7,Kerrisdale,3,18,3,138,23,251,3
8,Killarney,11,50,4,77,38,413,7
9,Kitsilano,20,92,10,254,61,1158,9


In [37]:
# I do not want the row containing grand_total
vpd_df = vpd_df.iloc[:24,:]
vpd_df

,Neighborhood,Sex Offences,Assaults,Robbery,Break & Enter,Auto Theft,Other Theft,Offensive Weapons
0,Arbutus Ridge,5,17,1,78,13,184,0
1,Central_Business_District,166,1819,258,798,185,10978,463
2,Dunbar-Southlands,6,16,2,85,13,264,2
3,Fairview,26,119,13,317,48,1769,17
4,Grandview-Woodland,31,236,35,292,122,1198,21
5,Hastings-Sunrise,27,118,12,197,85,916,7
6,Kensington-Cedar_Cottage,27,155,24,206,86,1088,22
7,Kerrisdale,3,18,3,138,23,251,3
8,Killarney,11,50,4,77,38,413,7
9,Kitsilano,20,92,10,254,61,1158,9


In [38]:
# make a function to calculate rate based on my formula
def cal_rate(df):
    '''Given a dataframe, iterate through all columns except the first, calculating the rate of each cell with respect to 
    the total sum of that column. Replace the rates with the original values and return the new dataframe.'''
    col_names = df.columns
    col_names = col_names[1:] # skip the first column because it is the neighborhood names column
    for col in col_names:
        total = df[col].sum()
        if total==0: # avoid fatal calculation of dividing by 0
            df[col] = 0
        else:
            df[col] = df[col]/total
    return df

In [39]:
# apply function to VPD
vpd_df = cal_rate(vpd_df)
vpd_df

<ipython-input-38-d6724bed9cef>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col]/total


,Neighborhood,Sex Offences,Assaults,Robbery,Break & Enter,Auto Theft,Other Theft,Offensive Weapons
0,Arbutus Ridge,0.009524,0.003778,0.001608,0.016613,0.009393,0.006241,0.000000
1,Central_Business_District,0.316190,0.404222,0.414791,0.169968,0.133671,0.372350,0.537123
2,Dunbar-Southlands,0.011429,0.003556,0.003215,0.018104,0.009393,0.008954,0.002320
3,Fairview,0.049524,0.026444,0.020900,0.067519,0.034682,0.060001,0.019722
4,Grandview-Woodland,0.059048,0.052444,0.056270,0.062194,0.088150,0.040634,0.024362
5,Hastings-Sunrise,0.051429,0.026222,0.019293,0.041960,0.061416,0.031069,0.008121
6,Kensington-Cedar_Cottage,0.051429,0.034444,0.038585,0.043876,0.062139,0.036903,0.025522
7,Kerrisdale,0.005714,0.004000,0.004823,0.029393,0.016618,0.008513,0.003480
8,Killarney,0.020952,0.011111,0.006431,0.016400,0.027457,0.014008,0.008121
9,Kitsilano,0.038095,0.020444,0.016077,0.054100,0.044075,0.039277,0.010441


In [40]:
# TPS next
tps_df

,Neighborhood,Sex Offences,Assaults,Robbery,Break & Enter,Auto Theft,Other Theft,Offensive Weapons
0,D11,67,480,71,88,41,863,75
1,D12,79,555,43,65,93,723,77
2,D13,64,407,65,124,53,463,43
3,D14,115,888,73,199,43,798,81
4,D22,81,601,81,115,32,852,92
5,D23,96,556,89,44,52,456,53
6,D31,135,976,106,113,60,816,140
7,D32,96,771,63,71,68,1087,101
8,D33,59,479,40,25,42,516,42
9,D41,120,896,90,136,34,1309,119


In [41]:
# apply function
tps_df = cal_rate(tps_df)
tps_df

,Neighborhood,Sex Offences,Assaults,Robbery,Break & Enter,Auto Theft,Other Theft,Offensive Weapons
0,D11,0.042894,0.039798,0.053788,0.044557,0.049220,0.060135,0.046904
1,D12,0.050576,0.046016,0.032576,0.032911,0.111645,0.050380,0.048155
2,D13,0.040973,0.033745,0.049242,0.062785,0.063625,0.032263,0.026892
3,D14,0.073624,0.073626,0.055303,0.100759,0.051621,0.055606,0.050657
4,D22,0.051857,0.049830,0.061364,0.058228,0.038415,0.059369,0.057536
5,D23,0.061460,0.046099,0.067424,0.022278,0.062425,0.031775,0.033146
6,D31,0.086428,0.080922,0.080303,0.057215,0.072029,0.056860,0.087555
7,D32,0.061460,0.063925,0.047727,0.035949,0.081633,0.075744,0.063164
8,D33,0.037772,0.039715,0.030303,0.012658,0.050420,0.035956,0.026266
9,D41,0.076825,0.074289,0.068182,0.068861,0.040816,0.091213,0.074422


In [42]:
# SPVM
spvm_df

,Neighborhood,Sex Offences,Assaults,Robbery,Break & Enter,Auto Theft,Other Theft,Offensive Weapons
0,1.0,22.0,81.0,17.0,69.0,66.0,202.0,3.0
1,3.0,50.0,394.0,40.0,175.0,86.0,286.0,6.0
2,4.0,25.0,167.0,31.0,74.0,60.0,190.0,2.0
3,5.0,44.0,284.0,50.0,188.0,175.0,603.0,7.0
4,7.0,67.0,440.0,73.0,271.0,377.0,756.0,5.0
5,8.0,58.0,522.0,52.0,148.0,69.0,411.0,10.0
6,9.0,11.0,89.0,15.0,68.0,45.0,179.0,2.0
7,11.0,44.0,447.0,83.0,174.0,88.0,494.0,10.0
8,13.0,62.0,527.0,77.0,211.0,99.0,724.0,4.0
9,26.0,82.0,660.0,144.0,363.0,258.0,1389.0,13.0


In [43]:
spvm_df = cal_rate(spvm_df)
spvm_df

,Neighborhood,Sex Offences,Assaults,Robbery,Break & Enter,Auto Theft,Other Theft,Offensive Weapons
0,1.0,0.011242,0.006102,0.007122,0.009866,0.015274,0.008459,0.008721
1,3.0,0.025549,0.029680,0.016757,0.025021,0.019903,0.011977,0.017442
2,4.0,0.012775,0.012580,0.012987,0.010580,0.013886,0.007957,0.005814
3,5.0,0.022483,0.021394,0.020947,0.026880,0.040500,0.025252,0.020349
4,7.0,0.034236,0.033145,0.030582,0.038747,0.087248,0.031660,0.014535
5,8.0,0.029637,0.039322,0.021785,0.021161,0.015969,0.017212,0.029070
6,9.0,0.005621,0.006704,0.006284,0.009723,0.010414,0.007496,0.005814
7,11.0,0.022483,0.033672,0.034772,0.024878,0.020366,0.020688,0.029070
8,13.0,0.031681,0.039699,0.032258,0.030169,0.022911,0.030320,0.011628
9,26.0,0.041901,0.049718,0.060327,0.051902,0.059708,0.058168,0.037791


Great!

🤠 __Merge the dataframes into one.__

In [44]:
# first add police name in front of each Neighoborhood name so that I can identify where each neighborhood came from
# VPD
vpd_df['Neighborhood'] = 'VPD ' + vpd_df['Neighborhood']
vpd_df

<ipython-input-44-beec367b603e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vpd_df['Neighborhood'] = 'VPD ' + vpd_df['Neighborhood']


,Neighborhood,Sex Offences,Assaults,Robbery,Break & Enter,Auto Theft,Other Theft,Offensive Weapons
0,VPD Arbutus Ridge,0.009524,0.003778,0.001608,0.016613,0.009393,0.006241,0.000000
1,VPD Central_Business_District,0.316190,0.404222,0.414791,0.169968,0.133671,0.372350,0.537123
2,VPD Dunbar-Southlands,0.011429,0.003556,0.003215,0.018104,0.009393,0.008954,0.002320
3,VPD Fairview,0.049524,0.026444,0.020900,0.067519,0.034682,0.060001,0.019722
4,VPD Grandview-Woodland,0.059048,0.052444,0.056270,0.062194,0.088150,0.040634,0.024362
5,VPD Hastings-Sunrise,0.051429,0.026222,0.019293,0.041960,0.061416,0.031069,0.008121
6,VPD Kensington-Cedar_Cottage,0.051429,0.034444,0.038585,0.043876,0.062139,0.036903,0.025522
7,VPD Kerrisdale,0.005714,0.004000,0.004823,0.029393,0.016618,0.008513,0.003480
8,VPD Killarney,0.020952,0.011111,0.006431,0.016400,0.027457,0.014008,0.008121
9,VPD Kitsilano,0.038095,0.020444,0.016077,0.054100,0.044075,0.039277,0.010441


In [45]:
# TPS
tps_df['Neighborhood'] = 'TPS ' + tps_df['Neighborhood']
tps_df

,Neighborhood,Sex Offences,Assaults,Robbery,Break & Enter,Auto Theft,Other Theft,Offensive Weapons
0,TPS D11,0.042894,0.039798,0.053788,0.044557,0.049220,0.060135,0.046904
1,TPS D12,0.050576,0.046016,0.032576,0.032911,0.111645,0.050380,0.048155
2,TPS D13,0.040973,0.033745,0.049242,0.062785,0.063625,0.032263,0.026892
3,TPS D14,0.073624,0.073626,0.055303,0.100759,0.051621,0.055606,0.050657
4,TPS D22,0.051857,0.049830,0.061364,0.058228,0.038415,0.059369,0.057536
5,TPS D23,0.061460,0.046099,0.067424,0.022278,0.062425,0.031775,0.033146
6,TPS D31,0.086428,0.080922,0.080303,0.057215,0.072029,0.056860,0.087555
7,TPS D32,0.061460,0.063925,0.047727,0.035949,0.081633,0.075744,0.063164
8,TPS D33,0.037772,0.039715,0.030303,0.012658,0.050420,0.035956,0.026266
9,TPS D41,0.076825,0.074289,0.068182,0.068861,0.040816,0.091213,0.074422


In [46]:
# SPVM
spvm_df['Neighborhood'] = spvm_df['Neighborhood'].map(str) # this was a float object - make it string
spvm_df['Neighborhood'] = 'SPVM ' + spvm_df['Neighborhood']
spvm_df

,Neighborhood,Sex Offences,Assaults,Robbery,Break & Enter,Auto Theft,Other Theft,Offensive Weapons
0,SPVM 1.0,0.011242,0.006102,0.007122,0.009866,0.015274,0.008459,0.008721
1,SPVM 3.0,0.025549,0.029680,0.016757,0.025021,0.019903,0.011977,0.017442
2,SPVM 4.0,0.012775,0.012580,0.012987,0.010580,0.013886,0.007957,0.005814
3,SPVM 5.0,0.022483,0.021394,0.020947,0.026880,0.040500,0.025252,0.020349
4,SPVM 7.0,0.034236,0.033145,0.030582,0.038747,0.087248,0.031660,0.014535
5,SPVM 8.0,0.029637,0.039322,0.021785,0.021161,0.015969,0.017212,0.029070
6,SPVM 9.0,0.005621,0.006704,0.006284,0.009723,0.010414,0.007496,0.005814
7,SPVM 11.0,0.022483,0.033672,0.034772,0.024878,0.020366,0.020688,0.029070
8,SPVM 13.0,0.031681,0.039699,0.032258,0.030169,0.022911,0.030320,0.011628
9,SPVM 26.0,0.041901,0.049718,0.060327,0.051902,0.059708,0.058168,0.037791


In [47]:
# concatenate into one dataframe
pol_df = pd.concat([vpd_df, tps_df, spvm_df], ignore_index=True)
pol_df

,Neighborhood,Sex Offences,Assaults,Robbery,Break & Enter,Auto Theft,Other Theft,Offensive Weapons
0,VPD Arbutus Ridge,0.009524,0.003778,0.001608,0.016613,0.009393,0.006241,0.000000
1,VPD Central_Business_District,0.316190,0.404222,0.414791,0.169968,0.133671,0.372350,0.537123
2,VPD Dunbar-Southlands,0.011429,0.003556,0.003215,0.018104,0.009393,0.008954,0.002320
3,VPD Fairview,0.049524,0.026444,0.020900,0.067519,0.034682,0.060001,0.019722
4,VPD Grandview-Woodland,0.059048,0.052444,0.056270,0.062194,0.088150,0.040634,0.024362
...,...,...,...,...,...,...,...,...
68,SPVM 42.0,0.025549,0.034652,0.032677,0.027023,0.067577,0.032958,0.040698
69,SPVM 45.0,0.021461,0.019887,0.020947,0.015013,0.035871,0.013192,0.020349
70,SPVM 46.0,0.021972,0.016271,0.019690,0.014870,0.027771,0.024708,0.008721
71,SPVM 48.0,0.074093,0.048814,0.049434,0.051187,0.058088,0.039114,0.029070


Amazing! We're done

In [48]:
# save this cleaned dataframe
pol_df.to_csv('./data/pol_df.csv')